<a href="https://colab.research.google.com/github/Chait314/reasoning/blob/main/Reasoning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Loading base models and environment

In [ ]:
import torch
import torch.nn as nn
from transformers import AutoModelForSequenceClassification, AutoTokenizer

In [ ]:
model_name = "Qwen/Qwen2-1.5B-Instruct";

In [ ]:
from datasets import load_dataset
import os

model = AutoModelForSequenceClassification.from_pretrained(model_name);
tokenizer = AutoTokenizer.from_pretrained(model_name);

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/660 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.09G [00:00<?, ?B/s]

Some weights of Qwen2ForSequenceClassification were not initialized from the model checkpoint at Qwen/Qwen2-1.5B-Instruct and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

In [ ]:
import pandas as pd

df = pd.read_json("hf://datasets/TIGER-Lab/MathInstruct/MathInstruct.json")

In [ ]:
print(df.head())

                            source  \
0           data/CoT/aqua_rat.json   
1           data/CoT/aqua_rat.json   
2  data/PoT/aqua_rat_filtered.json   
3            data/CoT/gsm_rft.json   
4  data/PoT/aqua_rat_filtered.json   

                                         instruction  \
0  The distance between two stars is 6.52 × 10^5 ...   
1  How many ways can the letters in the word COMM...   
2  A team of six entered for a shooting competiti...   
3  A psychiatrist has 4 patients that need 25 ses...   
4  The radius of a wheel is 22.4 cm. What is the ...   

                                              output  
0  Let's think about the multi-choice question.\n...  
1  Let's solve the multi-choice question step by ...  
2  answers = ['A', 'B', 'C', 'D', 'E']\n# If the ...  
3  The second patient needs 6+5 = 11 sessions\n25...  
4  radius = 22.4\nresolutions = 500\n# calculate ...  


In [ ]:
class ORM(nn.Module):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model;
        self.tokenizer = tokenizer;
        self.loss = nn.MSELoss();
        self.tokenizer.pad_token = self.tokenizer.eos_token;

    def Lorm(self, questions, answers, correctness_labels):
        inputs = self.tokenizer(
            questions,
            answers,
            return_tensors = "pt",
            padding = True
        );
        outputs = self.model(**inputs);

        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            outputs.logits.squeeze(),
            torch.tensor(correctness_labels).float()
        )
        return loss;

    def predict(self, questions, answers):
        inputs = self.tokenizer(
            questions, answers, padding = True, return_tensors = 'pt'
        )

        outputs = self.model(**inputs);

        return torch.sigmoid(outputs.logits);



In [ ]:
class PRM(nn.Module):
    def __init__(self, model, tokenizer):
        super().__init__()
        self.model = model;
        self.tokenizer = tokenizer;
        self.loss = nn.MSELoss();
        self.tokenizer.pad_token = self.tokenizer.eos_token;

    def Lprm(self, questions, partial_ans, step_labels):
        inputs = self.tokenizer(
            questions,
            partial_ans,
            return_tensors = "pt",
            padding = True
        );
        outputs = self.model(**inputs);

        loss = torch.nn.functional.binary_cross_entropy_with_logits(
            outputs.logits.squeeze(),
            torch.tensor(step_labels).float()
        )
        return loss;

    def score_step(self, questions, partial_ans):
        with torch.no_grad():
            inputs = self.tokenizer(
                questions,
                partial_ans,
                return_tensors = "pt",
                padding = True
            );
            return torch.sigmoid(self.model(**inputs).logits);


In [ ]:
print(df.columns);

Index(['source', 'instruction', 'output'], dtype='object')


In [ ]:
class PPO(nn.Module):
    def __init__(self, model, tokenizer,train_data):
        super().__init__()
        self.model = model;
        self.tokenizer = tokenizer;
        self.loss = nn.MSELoss();
        self.tokenizer = tokenizer;
        self.train_data = train_data;

    def clip_loss(reward_model, threshold):
        for i in range(len(reward_model)):
            if reward_model[i] > threshold:
                reward_model[i] = threshold;
            elif reward_model[i] < -threshold:
                reward_model[i] = -threshold;
            else:
                reward_model[i] = 0;
        return reward_model;

    def delta_loss(reward_model, threshold):
        for i in range(len(reward_model)):
            if i == len(reward_model)-1:
                reward_model[i] = 0;
            else:
                reward_model[i] = reward_model[i] - reward_model[i+1];
        return reward_model;

    def prepare_data_orm(dataset):
        samples = [];
        for data in range(dataset):
            solution = data['output'];
            steps = solution.split('\n');

            for i in range(len(steps)+1):
                samples.append({
                    "question": data['instruction'],
                    'output' : data['output'],
                    'correctness_label' : 1
                })
        return samples;

    def prepare_data_prm(dataset):
        samples = [];
        for data in range(dataset):
            solution = data['output'];
            steps = solution.split('\n');

            for i in range(len(steps)+1):
                prefix = "\n".join(steps[:i]);
                samples.append({
                    "question": data['instruction'],
                    'solution_prefix' : prefix,
                    'step_label' : 1
                })
        return samples;







